In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from IPython.core.pylabtools import figsize
from tqdm import tqdm

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

## Read Data

In [30]:
train_df = pd.read_csv('input/train.csv')
print(train_df.shape)
train_df.head(1)

(136950, 6)


,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63


In [31]:
test_df = pd.read_csv('input/test.csv')
test_df.shape

(27375, 5)

## Naive Modeling

In [32]:
def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [33]:
cats = ['country', 'store', 'product']

In [34]:
def pre_process(df, cats=cats):
    df['comb'] = df[cats].apply(lambda x: '|'.join(list(x)), axis=1)
    df['y'] = df['num_sold']
    df['ds'] = df['date']
    groups_by_comb = df.groupby('comb')
    print(len(groups_by_comb.groups.keys()))
    
    return df, groups_by_comb

In [35]:
train_df, groups_by_comb = pre_process(train_df)

75


In [50]:
smapes = []

for comb in train_df.comb.unique():
    country = comb.split('|')[0]
    group = groups_by_comb.get_group(comb)
    
    train = group[group['date'] < '2021-01-01']
    val = group[group['date'] > '2020-12-31']
    future = ExponentialSmoothing(train['y'], trend='mul').fit()
    forecast = future.forecast(365)
    date_idx = pd.date_range('2021-01-01', periods=365, freq='D')
    forecast.index = date_idx
    
    val_smape = smape(val['y'].values, forecast)
    print(comb, f"{val_smape:.2f}")
    
    smapes.append(val_smape)

/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Argentina|Kaggle Learn|Using LLMs to Improve Your Coding 45.71
Argentina|Kaggle Learn|Using LLMs to Train More LLMs 42.87


/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Argentina|Kaggle Learn|Using LLMs to Win Friends and Influence People 19.63
Argentina|Kaggle Learn|Using LLMs to Win More Kaggle Competitions 37.59


/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


...

/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Spain|Kagglazon|Using LLMs to Improve Your Coding 96.10
Spain|Kagglazon|Using LLMs to Train More LLMs 99.26


/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Spain|Kagglazon|Using LLMs to Win Friends and Influence People 14.15
Spain|Kagglazon|Using LLMs to Win More Kaggle Competitions 123.26
Spain|Kagglazon|Using LLMs to Write Better 108.02


/Users/xulian/anaconda3/envs/proph/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [51]:
np.mean(smapes)

53.09927672299071